In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use("ggplot")
from sklearn.feature_selection import chi2

In [2]:
# Load the data
allusers = pd.read_csv("user_table.csv", index_col="user_id")
allusers.head()

,date,device,sex
user_id,,,
450007,2015-02-28,Desktop,Female
756838,2015-01-13,Desktop,Male
568983,2015-04-09,Desktop,Male
190794,2015-02-18,Desktop,Female
537909,2015-01-15,Desktop,Male


In [3]:
user_to_search = pd.read_csv("search_page_table.csv", index_col="user_id")
user_to_search.head()

,page
user_id,
15866,search_page
347058,search_page
577020,search_page
780347,search_page
383739,search_page


In [4]:
user_to_pay = pd.read_csv("payment_page_table.csv", index_col="user_id")
user_to_pay.head()

,page
user_id,
253019,payment_page
310478,payment_page
304081,payment_page
901286,payment_page
195052,payment_page


In [5]:
user_to_confirm = pd.read_csv("payment_confirmation_table.csv", index_col="user_id")
user_to_confirm.head()

,page
user_id,
123100,payment_confirmation_page
704999,payment_confirmation_page
407188,payment_confirmation_page
538348,payment_confirmation_page
841681,payment_confirmation_page


In [6]:
allusers.loc[user_to_search.index, "page"] = user_to_search["page"]
allusers.head()

,date,device,sex,page
user_id,,,,
450007,2015-02-28,Desktop,Female,NaN
756838,2015-01-13,Desktop,Male,NaN
568983,2015-04-09,Desktop,Male,search_page
190794,2015-02-18,Desktop,Female,search_page
537909,2015-01-15,Desktop,Male,NaN


In [7]:
allusers.loc[user_to_pay.index, "page"] = user_to_pay["page"]
allusers.tail(10)

,date,device,sex,page
user_id,,,,
128231,2015-02-25,Desktop,Female,search_page
545678,2015-03-03,Desktop,Female,search_page
492404,2015-02-27,Desktop,Male,payment_page
352574,2015-02-23,Desktop,Female,NaN
163518,2015-02-26,Mobile,Female,payment_page
307667,2015-03-30,Desktop,Female,NaN
642989,2015-02-08,Desktop,Female,search_page
659645,2015-04-13,Desktop,Male,search_page
359779,2015-03-23,Desktop,Male,NaN


In [8]:
allusers.loc[user_to_confirm.index, "page"] = user_to_confirm["page"]

In [9]:
# Let's change the name of page in all users to a better name
allusers.rename(columns={"page":"final_page"}, inplace=True)
allusers["final_page"].fillna("home_page", inplace=True)

In [10]:
# change date object to pandas datetime object
allusers["date"] = pd.to_datetime(allusers["date"])
allusers.head()

,date,device,sex,final_page
user_id,,,,
450007,2015-02-28,Desktop,Female,home_page
756838,2015-01-13,Desktop,Male,home_page
568983,2015-04-09,Desktop,Male,search_page
190794,2015-02-18,Desktop,Female,search_page
537909,2015-01-15,Desktop,Male,home_page


In [11]:
allusers.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 90400 entries, 450007 to 438929
Data columns (total 4 columns):
date          90400 non-null datetime64[ns]
device        90400 non-null object
sex           90400 non-null object
final_page    90400 non-null object
dtypes: datetime64[ns](1), object(3)
memory usage: 3.4+ MB


In [12]:
allusers.to_csv("all_users.csv",index_label="user_id")

A full picture of funnel conversion rate for both desktop and mobile users

In [13]:
allusers.groupby("device")["final_page"].apply(lambda s:s.value_counts()).unstack()

,home_page,search_page,payment_page,payment_confirmation_page
device,,,,
Desktop,30100,27090,2860,150
Mobile,15100,12080,2718,302


In [14]:
allusers.groupby('device')["final_page"].apply(lambda s: s.value_counts(normalize=True)).unstack()

,home_page,search_page,payment_page,payment_confirmation_page
device,,,,
Desktop,0.5,0.45,0.047508,0.002492
Mobile,0.5,0.40,0.090000,0.010000


Some insights on what the product team should focus on in order to improve the conversion rate as well as anything you might discover that could help improve conversion rate.

In [15]:
allusers.head()

,date,device,sex,final_page
user_id,,,,
450007,2015-02-28,Desktop,Female,home_page
756838,2015-01-13,Desktop,Male,home_page
568983,2015-04-09,Desktop,Male,search_page
190794,2015-02-18,Desktop,Female,search_page
537909,2015-01-15,Desktop,Male,home_page


In [16]:
allusers.groupby("sex")["final_page"].value_counts()

sex     final_page               
Female  home_page                    22399
        search_page                  19576
        payment_page                  2859
        payment_confirmation_page      241
Male    home_page                    22801
        search_page                  19594
        payment_page                  2719
        payment_confirmation_page      211
Name: final_page, dtype: int64

In [17]:
X = allusers.copy()

In [18]:
X["device"].value_counts()

Desktop    60200
Mobile     30200
Name: device, dtype: int64

In [19]:
X["from_mobile"] = (X["device"] == "Mobile").astype(int)
del X["device"]
# for simplicity let's drop date
del X["date"]

In [20]:
X["is_male"] = (X["sex"] == "Male").astype(int)
del X["sex"]

In [21]:
X["converted"] = (X["final_page"] == "payment_confirmation_page").astype(int)
del X["final_page"]

In [22]:
X["converted"].mean() * 100 # highly imbalanced classification problem

0.5

In [23]:
X.head()

,from_mobile,is_male,converted
user_id,,,
450007,0,0,0
756838,0,1,0
568983,0,1,0
190794,0,0,0
537909,0,1,0


In [24]:
X.describe()

,from_mobile,is_male,converted
count,90400.000000,90400.000000,90400.000000
mean,0.334071,0.501383,0.005000
std,0.471667,0.500001,0.070534
min,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000
50%,0.000000,1.000000,0.000000
75%,1.000000,1.000000,0.000000
max,1.000000,1.000000,1.000000


In [25]:
# impact of sex
X.groupby("is_male")["converted"].agg(["count", "mean"]).sort_values(by="mean", ascending=False)

,count,mean
is_male,,
0,45075,0.005347
1,45325,0.004655


In [26]:
# Statistical Significance
X = X
y = X.pop("converted")

In [27]:
scores, pvalues = chi2(X,y)

In [28]:
pd.DataFrame({"chi2_score":scores, "p-value":pvalues}, index=X.columns).sort_values(by="chi2_score", ascending=False)

,chi2_score,p-value
from_mobile,151.758794,7.153913e-35
is_male,1.082703,2.980938e-01


In [29]:
from sklearn.tree import DecisionTreeClassifier, export_graphviz
dt = DecisionTreeClassifier()
dt.fit(X,y)
dot_data = export_graphviz(dt, out_file=None, 
                           feature_names=X.columns, 
                           class_names=["NotConverted", "Converted"],
                           proportion=True, 
                           leaves_parallel=True, filled=True)